<div style="text-align: center;">
    <h1><strong>Normalization Calculator</strong></h1>
</div>

#### Imports

In [ ]:
import numpy as np
import cv2
import os
import time

#### Colors & Paths

In [ ]:
CYAN, GREEN, YELLOW, BLUE, RED, RESET = "\033[96m", "\033[92m", "\033[93m", "\033[94m", "\033[91m", "\033[0m"
IMAGES_PATH = r'data/EuroSATsplit'

#### Calculation

In [ ]:
pixel_sum, pixel_squared_sum, pixel_count = np.zeros(3, dtype=np.float64), np.zeros(3, dtype=np.float64), 0
files_list = [os.path.join(root, file) for root, _, files in os.walk(IMAGES_PATH) for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

total_samples = len(files_list)
start_time = time.time()

for i, file_path in enumerate(files_list, 1):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        img_resized = cv2.resize(img, (64, 64))
        pixel_sum += np.sum(img_resized, axis=(0, 1))
        pixel_squared_sum += np.sum(img_resized ** 2, axis=(0, 1))
        pixel_count += img_resized.shape[0] * img_resized.shape[1]

    elapsed_time = time.time() - start_time
    seconds_per_sample = elapsed_time / i
    eta = (total_samples - i) * seconds_per_sample
    eta_h, eta_m, eta_s = int(eta // 3600), int((eta % 3600) // 60), int(eta % 60)

    print(f"{CYAN}Processing:{RESET} {GREEN}{i}/{total_samples}{RESET} files "
          f"({YELLOW}{elapsed_time:.2f}{RESET} sec elapsed, {BLUE}{seconds_per_sample:.2f}{RESET} sec/file, "
          f"ETA: {RED}{eta_h}h {GREEN}{eta_m}m {BLUE}{eta_s}s{RESET})", end="\r")

print()

if pixel_count > 0:
    mean_np = pixel_sum / pixel_count
    std_np = np.sqrt(pixel_squared_sum / pixel_count - mean_np ** 2)
    std_np = np.maximum(std_np, 1e-6)  # Prevent extreme contrast
else:
    mean_np, std_np = np.zeros(3), np.ones(3)  # Default to safe values if no images processed

with open(r'txts/normalization.txt', 'w') as file:
    file.write(f"mean={mean_np.tolist()}, std={std_np.tolist()}")

print(f"\n{GREEN}Processing complete!{RESET} Mean and std saved to 'normalization.txt'")